# FAER Agent Prototyping Workbench

This notebook provides an interactive environment for developing and testing clinical shadow agents.

## Contents
1. Setup & Imports
2. Running Simulations
3. Creating MetricsSummary
4. Testing Agents
5. Exploring Insights
6. Custom Rule Development

## 1. Setup & Imports

In [ ]:
# Standard imports
import sys
from pathlib import Path

# Add src to path if needed
project_root = Path.cwd().parent
if str(project_root / 'src') not in sys.path:
    sys.path.insert(0, str(project_root / 'src'))

# Core imports
import numpy as np
import pandas as pd
from pprint import pprint

In [ ]:
# FAER simulation imports
from faer.core.scenario import FullScenario
from faer.model.full_model import run_full_simulation
from faer.experiment.runner import multiple_replications

In [ ]:
# Agent imports
from faer.agents import (
    HeuristicShadowAgent,
    AgentOrchestrator,
    OrchestratorConfig,
    MetricsSummary,
    ClinicalThreshold,
    Severity,
    InsightCategory,
    NHS_THRESHOLDS,
)

print("Imports successful!")

## 2. Running Simulations

Let's run some simulations to generate data for agent testing.

In [ ]:
# Create a baseline "healthy" scenario
healthy_scenario = FullScenario(
    run_length=480.0,      # 8 hours
    warm_up=60.0,          # 1 hour warm-up
    n_triage=2,
    n_ed_bays=8,
    arrival_rate=6.0,      # 6 patients/hour
    random_seed=42,
)

print(f"Healthy scenario: {healthy_scenario.n_ed_bays} ED bays, {healthy_scenario.arrival_rate}/hr arrivals")

In [ ]:
# Create a "stressed" scenario
stressed_scenario = FullScenario(
    run_length=480.0,
    warm_up=60.0,
    n_triage=1,            # Understaffed
    n_ed_bays=4,           # Limited bays
    arrival_rate=10.0,     # High demand
    random_seed=42,
)

print(f"Stressed scenario: {stressed_scenario.n_ed_bays} ED bays, {stressed_scenario.arrival_rate}/hr arrivals")

In [ ]:
# Run multiple replications
print("Running healthy scenario (5 reps)...")
healthy_results = multiple_replications(healthy_scenario, n_reps=5)
print(f"  Arrivals: {np.mean(healthy_results['arrivals']):.1f}")
print(f"  Mean treatment wait: {np.mean(healthy_results['mean_treatment_wait']):.1f} min")

print("\nRunning stressed scenario (5 reps)...")
stressed_results = multiple_replications(stressed_scenario, n_reps=5)
print(f"  Arrivals: {np.mean(stressed_results['arrivals']):.1f}")
print(f"  Mean treatment wait: {np.mean(stressed_results['mean_treatment_wait']):.1f} min")

## 3. Creating MetricsSummary

Convert raw results to the standardized MetricsSummary format.

In [ ]:
# Convert to MetricsSummary
healthy_metrics = MetricsSummary.from_run_results(healthy_results, "Healthy Baseline")
stressed_metrics = MetricsSummary.from_run_results(stressed_results, "Stressed System")

print("Healthy Metrics:")
print(f"  Arrivals: {healthy_metrics.arrivals:.0f}")
print(f"  Mean treatment wait: {healthy_metrics.mean_treatment_wait:.1f} min")
print(f"  P95 treatment wait: {healthy_metrics.p95_treatment_wait:.1f} min")
print(f"  ED utilization: {healthy_metrics.util_ed_bays:.1%}")
print(f"  P(delay): {healthy_metrics.p_delay:.1%}")

print("\nStressed Metrics:")
print(f"  Arrivals: {stressed_metrics.arrivals:.0f}")
print(f"  Mean treatment wait: {stressed_metrics.mean_treatment_wait:.1f} min")
print(f"  P95 treatment wait: {stressed_metrics.p95_treatment_wait:.1f} min")
print(f"  ED utilization: {stressed_metrics.util_ed_bays:.1%}")
print(f"  P(delay): {stressed_metrics.p_delay:.1%}")

## 4. Testing Agents

Run the HeuristicShadowAgent against our scenarios.

In [ ]:
# Create agent
agent = HeuristicShadowAgent()

print(f"Agent: {agent.name}")
print(f"Description: {agent.description}")
print(f"Health check: {agent.health_check()}")

In [ ]:
# Analyze healthy scenario
healthy_insights = agent.analyze(healthy_metrics)

print(f"Healthy scenario insights: {len(healthy_insights)}")
for insight in healthy_insights:
    print(f"  [{insight.severity.value}] {insight.title}")

In [ ]:
# Analyze stressed scenario
stressed_insights = agent.analyze(stressed_metrics)

print(f"Stressed scenario insights: {len(stressed_insights)}")
for insight in stressed_insights:
    print(f"  [{insight.severity.value}] {insight.title}")

## 5. Exploring Insights

Dive deeper into the insights generated.

In [ ]:
# Detailed view of each insight
for i, insight in enumerate(stressed_insights, 1):
    print(f"\n{'='*60}")
    print(f"INSIGHT {i}: {insight.title}")
    print(f"{'='*60}")
    print(f"Severity: {insight.severity.value}")
    print(f"Category: {insight.category.value}")
    print(f"\nMessage:\n{insight.message}")
    print(f"\nEvidence:")
    for k, v in insight.evidence.items():
        print(f"  {k}: {v}")
    print(f"\nRecommendation:\n{insight.recommendation}")

In [ ]:
# Using the orchestrator
orchestrator = AgentOrchestrator()
orchestrator.register(agent)

result = orchestrator.run_all(stressed_metrics)

print("Orchestrator Summary:")
pprint(result.summary)

In [ ]:
# Filter by severity
critical = result.get_critical_insights()
high_and_critical = result.get_high_and_critical()

print(f"Critical insights: {len(critical)}")
print(f"High + Critical insights: {len(high_and_critical)}")

## 6. Custom Rule Development

Create and test custom clinical threshold rules.

In [ ]:
# View existing NHS thresholds
print("NHS Thresholds:")
for rule in NHS_THRESHOLDS:
    print(f"  {rule.title}: {rule.metric} {rule.operator} {rule.threshold} -> {rule.severity.value}")

In [ ]:
# Create custom thresholds
custom_thresholds = [
    ClinicalThreshold(
        metric="p95_treatment_wait",
        threshold=120.0,  # Stricter 2-hour target
        operator="gt",
        severity=Severity.HIGH,
        category=InsightCategory.WAIT_TIME,
        title="2-Hour Wait Standard Exceeded",
        message_template=(
            "P95 treatment wait of {value:.0f} minutes exceeds our 2-hour target. "
            "This is stricter than NHS standards but aligns with our quality goals."
        ),
        recommendation="Review patient flow and consider additional resources.",
    ),
    ClinicalThreshold(
        metric="util_ed_bays",
        threshold=0.70,  # Earlier warning
        operator="gt",
        severity=Severity.MEDIUM,
        category=InsightCategory.CAPACITY,
        title="ED Capacity Warning",
        message_template=(
            "ED bay utilization at {value:.0%}. While not critical, "
            "this is approaching levels where congestion may develop."
        ),
        recommendation="Monitor closely. Consider proactive discharge planning.",
    ),
]

print("Custom thresholds created:")
for rule in custom_thresholds:
    print(f"  {rule.title}")

In [ ]:
# Test custom agent
custom_agent = HeuristicShadowAgent(thresholds=custom_thresholds)

custom_insights = custom_agent.analyze(healthy_metrics)

print(f"Custom agent insights on healthy scenario: {len(custom_insights)}")
for insight in custom_insights:
    print(f"  [{insight.severity.value}] {insight.title}")

In [ ]:
# Combine NHS + custom thresholds
combined_agent = HeuristicShadowAgent(thresholds=NHS_THRESHOLDS + custom_thresholds)

combined_insights = combined_agent.analyze(stressed_metrics)

print(f"Combined agent insights on stressed scenario: {len(combined_insights)}")
for insight in combined_insights:
    print(f"  [{insight.severity.value}] {insight.title}")

## 7. Manual Metrics Testing

Create synthetic metrics to test specific edge cases.

In [ ]:
# Create a scenario with specific metrics to trigger compound rules
compound_test_metrics = MetricsSummary(
    scenario_name="compound_test",
    run_timestamp="2026-01-10T12:00:00",
    n_replications=1,
    arrivals=300,
    arrivals_by_priority={"P1": 30, "P2": 90, "P3": 120, "P4": 60},  # 10% P1
    arrivals_by_mode={"ambulance": 200, "helicopter": 20, "walk_in": 80},
    mean_triage_wait=15.0,
    mean_treatment_wait=90.0,  # High - should trigger compound with P1
    p95_treatment_wait=180.0,  # Under 4-hour threshold
    mean_system_time=200.0,
    p95_system_time=350.0,
    p_delay=0.40,
    util_triage=0.30,  # Low triage but high treatment wait
    util_ed_bays=0.80,
    util_itu=0.88,  # High ITU
    util_ward=0.75,
    util_theatre=0.75,  # High theatre - should trigger compound with ITU
    itu_admissions=25,
    mean_itu_wait=45.0,
    ward_admissions=80,
    mean_ward_wait=30.0,
    theatre_admissions=15,
    mean_theatre_wait=60.0,
    mean_boarding_time=35.0,
    p_boarding=0.20,
    mean_handover_delay=25.0,
    max_handover_delay=60.0,
)

# Analyze
compound_insights = agent.analyze(compound_test_metrics)

print("Compound rule testing:")
for insight in compound_insights:
    print(f"  [{insight.severity.value}] {insight.title}")
    if insight.category == InsightCategory.COMPOUND_RISK or insight.title.startswith("High Acuity"):
        print(f"    -> This is a compound rule!")

## Next Steps

Ideas for extending the agent layer:

1. **LLM Integration**: Replace heuristic logic with LLM analysis
2. **Capacity Advisor Agent**: Recommend specific capacity changes
3. **Scenario Comparator**: Compare metrics across scenarios
4. **Historical Memory**: Track patterns across multiple runs
5. **Custom Visualization**: Create dashboards for insights